In [ ]:
import sys
import os

package_path = '..'
if package_path not in sys.path:
    sys.path.append(package_path)

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os

from roi_rectangle import RoiRectangle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
from tqdm import tqdm

In [ ]:
from lmfit import Model, create_params
from lmfit.models import GaussianModel

mod = GaussianModel()

In [ ]:
from scipy.special import erf


def damp_osci(x, x0, oa, rg, ro, phase, sigma, w):
    damp = oa * np.exp(-(x-x0)/rg)*np.cos((x-x0)/ro-phase)
    err_func = -0.5 * (erf(-1/np.sqrt(2)*(x-x0)/sigma - sigma/w) ) + 0.5
    return damp * err_func


def exp_decay(x, Amplitude, x0, sigma, w, oa, rg, ro, phase):
    fit1 = Amplitude/2*np.exp(-(x-x0)/w + ((sigma/w)**2)/2)*(1-erf(-1/np.sqrt(2)*(x-x0)/sigma - sigma/w))
    osc = damp_osci(x, x0, oa, rg, ro, phase, sigma, w)
    return fit1 + osc


def gaussian(x, amplitude, center, sigma, offset):
    return amplitude * np.exp(-np.power(x - center, 2.) / (2 * np.power(sigma, 2.))) + offset

mymodel = Model(gaussian)
gmodel = Model(exp_decay)

In [ ]:
print(f'parameter names: {gmodel.param_names}')
print(f'independent variables: {gmodel.independent_vars}')

In [ ]:
params = gmodel.make_params(
    Amplitude=dict(value=0.06), 
    x0=dict(value=0.0), 
    sigma=dict(value=0.1), 
    w=dict(value=0.1), 
    oa=dict(value=0, vary=False),
    rg=dict(value=10, vary=False), 
    ro=dict(value=10, vary=False), 
    phase=dict(value=0, vary=False)
)
params

In [ ]:
from src.analyzer.loader import NpzLoader
from src.config.config import load_config

In [ ]:
config = load_config()
print("load_dir:", config.path.load_dir)
print("analysis_dir:", config.path.analysis_dir)

In [ ]:
npz_dir: str = config.path.npz_dir

run_n: int = 43
scan_n: int = 1
npz_file = os.path.join(npz_dir, f"run={run_n:0>4}_scan={scan_n:0>4}.npz")
_data = NpzLoader(npz_file).data
for key, val in _data.items():
    print(key)
    print(val.shape)

In [ ]:
poff_images = _data["poff"]
pon_images = _data["pon"]
delays = _data["delay"]

In [ ]:
roi_rect: RoiRectangle = RoiRectangle(x1=310, y1=217, x2=552, y2=436)
roi_poffs = roi_rect.slice(poff_images)
roi_pons = roi_rect.slice(pon_images)

roi_poffs.shape

In [ ]:
params = mymodel.make_params(
    amplitude=dict(value=100), 
    center=dict(value=100), 
    sigma=dict(value=5),
    offset=dict(value=5)
)
params

In [ ]:
image = roi_pons[-20]
x = np.arange(image.shape[1])
y = np.arange(image.shape[0])
sum_0 = image.sum(0)
sum_1 = image.sum(1)
result_x = gmodel.fit(sum_0, params, x=x)

In [ ]:
result_x.fit_report()

In [ ]:
plt.scatter(x, result_x.init_fit, label='initial fit')
plt.scatter(x, result_x.best_fit, label='best fit')
plt.scatter(x, sum_0, label='raw data')
plt.legend()

In [ ]:
x0s = []
y0s = []
for image in roi_pons:
    x = np.arange(image.shape[1])
    y = np.arange(image.shape[0])
    sum_0 = image.sum(0)
    sum_1 = image.sum(1)
    result_x = mymodel.fit(sum_0, params, x=x)
    result_y = mymodel.fit(sum_1, params, x=y)

    x0 = result_x.params['center'].value
    x0s.append(x0)
    y0 = result_y.params['center'].value
    y0s.append(y0)

In [ ]:
plt.plot(delays, x0s)

In [ ]:
plt.plot(delays, y0s)